In [2]:
df = spark.read.parquet("Files/Cotacoes/Novos/*.parquet")

df.createOrReplaceTempView("df")

display(df)

StatementMeta(, 59f2217b-f335-490a-b525-db64707a7779, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cf8a6939-e84c-46fd-806e-1397e67ece4c)

In [5]:
df = spark.sql("""
    SELECT 
        cotacaoCompra AS Cotacao,
        CAST(dataHoraCotacao as DATE) AS Data,
        moeda AS Moeda
    FROM
        df
    ORDER BY Data ASC
""").dropDuplicates(["Moeda","Data"])
display(df)

StatementMeta(, 59f2217b-f335-490a-b525-db64707a7779, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 67f140c7-fd21-476e-9e4e-0084d2e41c86)

In [11]:
%%sql

CREATE TABLE IF NOT EXISTS cotacoes(
    Cotacao DOUBLE,
    Data DATE,
    Moeda STRING
)
USING DELTA
PARTITIONED BY (Moeda)

StatementMeta(, 59f2217b-f335-490a-b525-db64707a7779, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [12]:
df.createOrReplaceTempView("df_novos")

spark.sql("""
    MERGE INTO cotacoes AS e 
    USING(
        SELECT
            Cotacao,
            Data,
            Moeda
        FROM
            df_novos
    ) as n 
    ON e.Moeda = n.Moeda
        AND e.Data = n.Data
    WHEN NOT MATCHED THEN 
        INSERT(Cotacao, Data, Moeda)
        VALUES(n.Cotacao, n.Data,n.Moeda)

""")


StatementMeta(, 59f2217b-f335-490a-b525-db64707a7779, 14, Finished, Available, Finished)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [14]:
%%sql
SELECT COUNT(*) FROM cotacoes

StatementMeta(, 59f2217b-f335-490a-b525-db64707a7779, 16, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [15]:
from notebookutils import mssparkutils

origem = "abfss://7e59932b-4cc3-434a-a636-66b2d74c36ed@onelake.dfs.fabric.microsoft.com/ef329037-45f5-42ae-a499-910a2b8086e0/Files/Cotacoes/Novos" 
destino = "abfss://7e59932b-4cc3-434a-a636-66b2d74c36ed@onelake.dfs.fabric.microsoft.com/ef329037-45f5-42ae-a499-910a2b8086e0/Files/Cotacoes/Carregados/"

if not mssparkutils.fs.exists(destino):
    mssparkutils.fs.mkdirs(destino)

arquivos = mssparkutils.fs.ls(origem)

for arquivo in arquivos:
    caminho_origem = arquivo.path
    nome_arquivo = arquivo.name
    caminho_destino = f"{destino}{nome_arquivo}"
    
    mssparkutils.fs.mv(caminho_origem, caminho_destino)

StatementMeta(, 59f2217b-f335-490a-b525-db64707a7779, 17, Finished, Available, Finished)